In [1]:
!pip install sacrebleu
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 820.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 1.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 2.0 MB/s eta 0:00:00a 0:00:01


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import sacrebleu

In [3]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("KapitalK/mGPT")
base_model = AutoModelForCausalLM.from_pretrained("ai-forever/mGPT")
model = PeftModel.from_pretrained(base_model, "KapitalK/mGPT")

adapter_config.json:   0%|          | 0.00/495 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.45G [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/66.8k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(filename, map_location=t

In [5]:
# # Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/mGPT")
model = AutoModelForCausalLM.from_pretrained("ai-forever/mGPT")

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.89M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

In [6]:
df = pd.read_csv("/kaggle/input/mansi-russian-parralel-corpus/overall_80K.csv", index_col=0)
df['to'] = '__mns__'
df['from'] = '__ru__'

In [7]:
df_other = df.copy()
df_other['to'] = '__ru__'
df_other['from'] = '__mns__'
df_other['source'], df_other['target'] = df_other['target'], df_other['source']

# Concatenate the original df and df_other
df = pd.concat([df, df_other], ignore_index=True)

In [8]:
train_df, test_val_df = train_test_split(df, test_size=0.2, random_state=42,shuffle=True, stratify=df['to'])
val_df, test_df = train_test_split(test_val_df, test_size=0.5, random_state=42, shuffle=True, stratify=test_val_df['to'])

In [30]:
# just for fast testing
a, b = train_test_split(test_df, test_size=0.006, random_state=42, shuffle=True, stratify=test_df['to'])

In [32]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

b = Dataset.from_pandas(b)

In [11]:
tokenizer.add_special_tokens({'additional_special_tokens': ['__mns__', '__ru__']})
model.resize_token_embeddings(len(tokenizer))

Embedding(100002, 2048)

In [12]:
def preprocess_function(examples):
    #examples['source'] = [f'{tok_from} {inp}' for tok_from, inp in zip(examples['from'], examples['source'])]
    #examples['target'] = [f'{tok_from} {inp}' for tok_from, inp in zip(examples['to'], examples['target'])]
    
    inputs = tokenizer(examples['source'], truncation=True, padding='max_length', max_length=128)
    targets = tokenizer(examples['target'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = targets['input_ids']
    return inputs

In [33]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# b = b.map(preprocess_function, batched=True)

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

In [14]:
train_dataset = train_dataset.remove_columns(['source', 'target', '__index_level_0__', 'to', 'from'])
val_dataset = val_dataset.remove_columns(['source', 'target', '__index_level_0__', 'to', 'from'])
test_dataset = test_dataset.remove_columns(['source', 'target', '__index_level_0__', 'to', 'from'])

In [15]:
import torch
torch.cuda.empty_cache()

In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./mGPT_finetuned_biling",
    eval_strategy="no",  # отключаем валидацию 
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2, 
    weight_decay=0.01,
    save_total_limit=1, 
    save_strategy="epoch",  # сохраняем токо после каждой эпохи
    num_train_epochs=2,
    max_steps=100,
    predict_with_generate=True,
    logging_dir="./logs",
    report_to="none", 
    fp16=True, 
    load_best_model_at_end=False,  # вырубаем загрузку лучшей модели в конце
)

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # убираем токены pad
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # вычисление метрик
    bleu = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels])
    chrf = sacrebleu.corpus_chrf(decoded_preds, [decoded_labels])

    return {"bleu": bleu.score, "chrf": chrf.score}

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

# Тренировка модели
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
500,22.859600
1000,19.088000
1500,16.549100
2000,12.864700
2500,8.083200
3000,6.149800
3500,5.711900
4000,5.428600
4500,5.261700
5000,5.117000


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=11400, training_loss=7.259199400784676, metrics={'train_runtime': 25838.5763, 'train_samples_per_second': 7.059, 'train_steps_per_second': 0.441, 'total_flos': 1.6929842565061018e+17, 'train_loss': 7.259199400784676, 'epoch': 1.4048059149722736})

In [17]:
import shutil
from IPython.display import FileLink

shutil.make_archive("mGPT_finetuned_biling", 'zip', "/kaggle/working/mGPT_finetuned_biling/")

'/kaggle/working/mGPT_finetuned_biling.zip'

In [18]:
FileLink('mGPT_finetuned_biling.zip')

/kaggle/working/mGPT_finetuned_biling.zip

In [38]:
import random
from tqdm import tqdm

tokenizer.src_lang = "__ru__"
tokenizer.tgt_lang = "__mns__"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

random.seed(42) 
# random_indices = random.sample(range(len(test_dataset)), 1000)
random_test_samples = b

def generate_translation(sample):
    input_ids = torch.tensor(sample['input_ids']).unsqueeze(0).to(device)  # Конвертируем список в тензор
    with torch.no_grad():
        generated_ids = model.generate(input_ids, max_length=200, num_beams=5, early_stopping=True)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

original_texts = []
correct_translations = []
model_translations = []

for sample in tqdm(random_test_samples):
    input_text = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
    correct_translation = tokenizer.decode(sample['labels'], skip_special_tokens=True)
    model_translation = generate_translation(sample)

    original_texts.append(input_text)
    correct_translations.append(correct_translation)
    model_translations.append(model_translation)
    
df_results = pd.DataFrame({
    "Original Text": original_texts,
    "Correct Translation": correct_translations,
    "Model Translation": model_translations
})

  0%|          | 0/98 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/peft/peft_model.py:1685: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")
100%|██████████| 98/98 [07:39<00:00,  4.69s/it]


In [37]:
df_results.loc[:, 'Original Text'] = df_results['Original Text'].str.replace('__en__', '')
df_results.loc[:, 'Correct Translation'] = df_results['Correct Translation'].str.replace('__en__', '')
df_results.loc[:, 'Model Translation'] = df_results['Model Translation'].str.replace('__en__', '')
    
bleu_score = sacrebleu.corpus_bleu(df_results['Model Translation'].tolist(), 
                                   [df_results['Correct Translation'].tolist()]).score
chrf_score = sacrebleu.corpus_chrf(df_results['Model Translation'].tolist(), 
                                   [df_results['Correct Translation'].tolist()]).score

print(f"BLEU Score: {bleu_score}")
print(f"ChrF Score: {chrf_score}")

df_results.head(50)

BLEU Score: 0.0
ChrF Score: 0.0


,Original Text,Correct Translation,Model Translation
0,"В это время, устроенное заседание называют «п...",Ты порат ва̄рыглым заседание «промежуточныиг»...,იციპალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტ...
1,В районе озера Чусовское обнаружены плутоний-...,"Чусовское тӯрт -238, -239, -240 плутоний хо̄н...",იციპალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტ...
2,"Волк слышал это, пришел к сараю и запел под г...","Ха̄йтнут ти латӈыт пуссын хулсанэ, сарай поха...",იციპალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტ...
3,Там одежду или другие вещи хорошо рассмотрят.,Тот маснут манос мо̄т пормас ёмащакв сунсыглаве.,იციპალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტ...
4,Е.А. Нёмысова тав ва̄т нупыл ат пор- мас музе...,"Е.А. Нёмысова, она в музей подарила 25 издели...",იციპალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტ...
5,Эвелина Борисова то̄р-ло̄мтыл нё̄тнэ сакв-ква...,Эвелина Борисова с куска ткани сделала красив...,იციპალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტ...
6,Пыгрищит Тимофей Молданов ёт тӯлыглапыт суссы...,"Юноши с Тимофей Молданов показывали сценки, о...",იციპალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტ...
7,Ва̄спун ёмас подускан,На подушку с утиными перьями,იციპალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტ...
8,В окошко глянь.,Ӣснасн а̄ӈкваталэн.,იციპალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტ...
9,"407.Саша анум дежуритан канумныл пентыстэ, ту...","407.Сдаю дежурство, а сам лезу к штурвалу.",იციპალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტალიტეტ...
